In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re as re
import json
from selenium import webdriver

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}
session = requests.Session()
session.headers.update(headers)

url = "https://krisha.kz/prodazha/kvartiry/"
r = session.get(url)

In [3]:
print(r.text)

<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8"/>
    <meta http-equiv="x-ua-compatible" content="ie=edge"/>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Продажа квартир 🏘 в Казахстане: купить квартиру – объявления на Крыше</title>
    <meta name="description" content="«Крыша» – крупнейшая база актуальных объявлений о ⭐ продаже квартир ⭐ в Казахстане на вашем компьютере. С нами легко купить квартиру мечты в Казахстане!"/>
                            <link rel="alternate" href="https://krisha.kz/prodazha/kvartiry/" hreflang="ru"/>
                    <link rel="alternate" href="https://krisha.kz/kz/prodazha/kvartiry/" hreflang="kz"/>
                            <link rel="canonical" href="https://krisha.kz/prodazha/kvartiry/kazakhstan/"/>
                    <link href="https://m.krisha.kz/prodazha/kvartiry/" media="only screen and (max-width: 640px)" rel="alternate" />
        <link rel="icon" type="image/x-icon" href="//krisha.kz/static/f

In [4]:
import time

soup = BeautifulSoup(r.text, "lxml")
cards = soup.select("div.a-card")
#cur = None

for card in cards:
    id = card["data-id"]
    url = f"https://krisha.kz/a/show/{id}"
    print("URL", url)
    #desc = card.select_one("div.a-card__text-preview").get_text(strip=True)

    r2 = session.get(url)
    #cur = r2.copy()
    print(r2.text)
    break

URL https://krisha.kz/a/show/699171233
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8"/>
    <meta http-equiv="x-ua-compatible" content="ie=edge"/>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Продажа 2-комнатной квартиры - №699171233: Егизбаева — Розыбакиевой, Алматы, Бостандыкский р-н — за 35000000 — Крыша</title>
    <meta name="description" content="Объявление о продаже 2-комнатной квартиры за 35000000 тенге в Алматы: Бостандыкский р-н, , Егизбаева — Розыбакиевой. Больше объявлений о продаже квартир в городе Алматы на сайте krisha.kz."/>
            <meta name="keywords" content="Продажа, 2-комнатной, квартиры, №699171233:, Егизбаева, Розыбакиевой, Алматы, Бостандыкский, 35000000, Крыша"/>
                            <link rel="alternate" href="https://krisha.kz/a/show/699171233" hreflang="ru"/>
                    <link rel="alternate" href="https://krisha.kz/kz/a/show/699171233" hreflang="kz"/>
                            <link rel="

In [5]:
soup2 = BeautifulSoup(r2.text, "lxml")
price_tag = soup2.select_one("div.offer__price")
print(price_tag)

<div class="offer__price">
                                                                                                                                                        35 000 000 <span class="currency-sign offer__currency">〒</span>
</div>


In [6]:
price = soup2.select_one("div.offer__price").get_text(strip=True)
#print(price)
price = price.replace("\xa0", "")
price = price.replace("〒", "")
print(price)
city = soup2.select_one("div.offer__location").get_text(strip=True)
city = re.search(r"^[^,]+", city).group()
print(city)

35000000
Алматы


In [7]:
import re

html = r2.text

lat = re.search(r'"lat"\s*:\s*([0-9.]+)', html)
lon = re.search(r'"lon"\s*:\s*([0-9.]+)', html)

if lat and lon:
    lat = float(lat.group(1))
    lon = float(lon.group(1))
    print(lat, lon)

43.230268675703 76.88924581268


In [8]:
price = soup2.select("div.offer__analytics")
print(price)

[<div class="offer__analytics">
<div class="a-analytics"></div>
</div>]


In [9]:
price_tag = soup.select_one("div.prices")

print(price_tag)



None


In [10]:
year = soup2.select("div.offer__advert-short-info")
u = 0

for i in year:
    text = i.get_text(strip=True)

    if u == 1:
        print(text)
    u += 1
    
    #print(text)
    continue

    floor = re.search(r"[0-9]*\sиз", text)

    if floor:
        max_floor = re.search(r"из\s[0-9]*", text).group()
        floor = floor.group()[:-3]
        max_floor = max_floor[3:]
        
        print(floor, max_floor, sep="|")

    #cur = i.get_text(strip=True)
    #value = re.search(r"[0-9]+", cur)
    #print(cur, value, sep=" | ")

#year = re.search(r"[0-9]+", year)
#print(year)

кирпичный


In [23]:
def prepare_entry(r2, id):
    result = []
    columns = []

    url = "https://krisha.kz/analytics/aPriceAnalysis/"
    params = {"id": id}
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://krisha.kz/a/show/762320257"
    }

    r = requests.get(url, params=params, headers=headers)

    soup3 = BeautifulSoup(r.text, "lxml")
    recommended = soup3.select("td.middle")

    rec_price = None
    for i in recommended:
        text = i.get_text()
        #print(text)
        buf = re.search(r"[0-9]+\s[0-9]+", text)
        if buf:
            rec_price = buf.group()
    rec_price = "".join(rec_price.split())

    if rec_price:
        result.append(rec_price)
        columns.append("rec_price_m")


    html = r2.text

    soup2 = BeautifulSoup(r2.text, "lxml")
    price = soup2.select_one("div.offer__price").get_text(strip=True)

    if price:
        price = price.replace("\xa0", "")
        price = price.replace("〒", "")
        result.append(price)
        columns.append("price")

    city = soup2.select_one("div.offer__location").get_text(strip=True)

    if city:
        city = re.search(r"^[^,]+", city).group()
        result.append(city)
        columns.append("city")

    lat = re.search(r'"lat"\s*:\s*([0-9.]+)', html)
    lon = re.search(r'"lon"\s*:\s*([0-9.]+)', html)

    if lat and lon:
        lat = lat.group(1)
        lon = lon.group(1)
        columns.append("lat")
        columns.append("lon")
        result.append(lat)
        result.append(lon)

    year = soup2.select("div.offer__advert-short-info")
    cnt = 0

    text = r2.text

    bathrooms = re.search(r"Санузел[0-9]", text)

    height = re.search(r"[0-9].[0-9]\sм</dd>", text)

    if bathrooms is None:
        bathrooms = re.search(r"[0-9]\sс/у", text)
        if bathrooms is None:
            bathrooms = 1
        else:
            bathrooms = bathrooms.group()[0]
    else:
        bathrooms = bathrooms.group()[-1]
    
    if height is None:
        height = 2
    else:
        height = height.group()[:-7]
    
    result.append(bathrooms)
    result.append(height)
    columns.append("bathrooms")
    columns.append("height")


    items = soup2.select("div.offer__info-item")

    text = " ".join(i.get_text(" ", strip=True) for i in items)
    text = text.replace("\xa0", " ")
    text = re.sub(r"\s+", " ", text).strip()

    state = re.search(r"квартиры.*", text, re.I)

    if state is None:
        state = "черновая отделка"
    else:
        state = state.group()[9:]

    result.append(state)
    columns.append("state")

    for i in year:
        cnt += 1
        text = i.get_text(strip=True)
        
        if cnt == 2:
            columns.append("type")
            result.append(text)

        m = re.search(r"\b(19|20)\d{2}\b", text)
        if m:
            columns.append("year")
            result.append(m.group())
            continue
        area = re.search(r"[0-9]*\sм²", text)
        if area:
            result.append(float(area.group()[:-3]))
            columns.append("area")
            continue
        floor = re.search(r"[0-9]*\sиз", text)
        if floor:
            max_floor = re.search(r"из\s[0-9]*", text)
        
        floor = re.search(r"[0-9]*\sиз", text)
        if floor:
            max_floor = re.search(r"из\s[0-9]*", text).group()
            floor = floor.group()[:-3]
            max_floor = max_floor[3:]
            columns.append("floor")
            columns.append("max_floor")
            result.append(floor)
            result.append(max_floor)
            continue
    
    return result, columns
        


In [21]:
test_url = "https://krisha.kz/a/show/762320257"
r2 = session.get(test_url)
an_url = "https://krisha.kz/analytics/aPriceAnalysis/"


results, cols = prepare_entry(r2, 762320257)
print(prepare_entry(r2, 762320257))
#print(a, b)

(['476830', '29000000', 'Астана', 51.161597100747, 71.457826886326, 1, '2.7', 'черновая отделка', 'кирпичный', '2013', '13', '15', 60.0], ['rec_price_m', 'price', 'city', 'lat', 'lon', 'bathrooms', 'height', 'state', 'type', 'year', 'floor', 'max_floor', 'area'])


In [13]:
items = soup2.select("div.offer__info-item")

text = " ".join(i.get_text(" ", strip=True) for i in items)
text = text.replace("\xa0", " ")
text = re.sub(r"\s+", " ", text).strip()

state = re.search(r"квартиры.*", text, re.I)

if state is None:
    state = "черновая отделка"
else:
    state = state.group()[9:]

print(state)

#print(text)
#print(state.group() if state else "не найдено")


не новый, но аккуратный ремонт Санузел совмещенный


In [14]:
import requests

url = "https://krisha.kz/analytics/aPriceAnalysis/"
params = {"id": 762320257}

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://krisha.kz/a/show/762320257"
}

r = requests.get(url, params=params, headers=headers)
#print(r.json())

soup3 = BeautifulSoup(r.text, "lxml")
recommended = soup3.select("td.middle")

rec_price = None
for i in recommended:
    text = i.get_text()
    print(text)
    buf = re.search(r"[0-9]+\s[0-9]+", text)
    if buf:
        rec_price = buf.group()
rec_price = "".join(rec_price.split())

#print(r.text)
#print(recommended)
print(rec_price)



476 830 〒 
Похожие квартиры в р-не Байконур

476830


In [17]:
cols = ['rec_price_m', 'price', 'city', 'lat', 'lon', 'bathrooms', 'height', 'state', 'type', 'year', 'floor', 'max_floor', 'area']
print(cols)

['rec_price_m', 'price', 'city', 'lat', 'lon', 'bathrooms', 'height', 'state', 'type', 'year', 'floor', 'max_floor', 'area']


In [24]:
import csv
import os
from pathlib import Path

file_path = Path("dataset.csv")

if not file_path.is_file():
    with open('dataset.csv', 'a') as file:
        file.write(" ".join(cols))

with open('dataset.csv', 'a') as file:
    file.writable(" ".join(results))


TypeError: sequence item 3: expected str instance, float found